In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
#import cufflinks as cf


# results

In [2]:
path_model = "results/20200509-161935self_learning_preprocessed/"
path_baseline = "results/20200509-194430self_learning_preprocessed/"
def get_stats(path, baseline = False):
    

    round_stats = pd.read_csv(path + "round_stats.csv", index_col=0)
    
    if baseline == False:
        columns = ["round_" + str(i) for i in range(9)]
        loss_summary = pd.read_csv(path + "loss_summary.csv",  names=columns, skiprows=1)
        residuals = pd.read_csv(path + "round_0/residuals.tsv",sep="\t", index_col=0)
        return round_stats, loss_summary, residuals
    else:
        return round_stats

round_stats_model, loss_summary, residuals = get_stats(path_model)
round_stats_baseline = get_stats(path_baseline, baseline = True)


In [3]:
round_stats_model

,best_valid_loss,epoch_mins,epoch_secs,test_loss,residual_loss,residual_mins,residual_secs,quantile,test_bleu,total_samples
0,1.663253,0,16,4.160481,6.713622,0,5,5.758230,NaN,15808
1,1.659017,0,16,4.008912,6.906172,0,11,5.944082,NaN,15936
2,1.869330,0,16,4.024507,6.993692,0,23,6.118941,NaN,16192
3,1.800914,0,17,3.848773,6.970763,0,47,6.102642,NaN,16640
4,2.046774,0,20,3.692813,6.842437,1,33,5.959591,NaN,17664
5,2.282257,0,24,3.419370,6.804133,3,12,5.778756,NaN,19648
6,2.609423,0,33,3.137922,6.812364,6,30,5.731229,NaN,23680
7,2.903279,0,55,2.910764,6.981319,7,36,5.879750,NaN,31680
8,3.165703,1,26,2.712317,7.726342,5,50,6.569008,NaN,40640
9,3.377806,1,54,2.655487,7.481101,4,32,6.548425,NaN,47296


In [4]:
round_stats_baseline

,best_valid_loss,epoch_mins,epoch_secs,test_loss,test_bleu,total_samples
0,1.661334,0,8,4.094427,NaN,15808
1,1.740563,0,8,4.073173,NaN,15936
2,1.968949,0,8,3.958783,NaN,16192
3,1.987877,0,9,3.913173,NaN,16640
4,2.270826,0,10,3.638732,NaN,17664
5,2.641845,0,12,3.494934,NaN,19648
6,3.102716,0,16,3.257977,NaN,23680
7,3.711199,0,25,3.055867,NaN,31680
8,4.182960,0,36,3.088721,NaN,40640
9,4.331377,0,46,3.092497,NaN,47296


In [5]:
round_stats_model["new_samples"] = round_stats_model["total_samples"].diff()
round_stats_baseline["new_samples"] = round_stats_baseline["total_samples"].diff()

wiki_total_nr = len(pd.read_csv("preprocessed_data/fb-wiki/wiki_dataset_cleaned_spelling.csv", index_col = 0))

first_5_rounds_percentage = round(100 * round_stats_model.new_samples[0:6].sum() / wiki_total_nr)
last_2_rounds_percentage = round(100 * round_stats_model.new_samples[:-1].sum() / wiki_total_nr)

first_5_rounds_percentage
#last_2_rounds_percentage




7.0

In [6]:
round_stats_model.new_samples[:9].sum() / wiki_total_nr

0.4664162283996995

In [7]:
round_stats_model[9:10]

,best_valid_loss,epoch_mins,epoch_secs,test_loss,residual_loss,residual_mins,residual_secs,quantile,test_bleu,total_samples,new_samples
9,3.377806,1,54,2.655487,7.481101,4,32,6.548425,NaN,47296,6656.0


In [15]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=round_stats_model.total_samples,y=round_stats_model.test_loss, name = "Self-Learning Model"))
fig.add_trace(go.Scatter(x=round_stats_baseline.total_samples,y=round_stats_baseline.test_loss, name = "Random-Pick Baseline Model"))
fig.add_trace(go.Scatter(
    x=[18200, 47500, 61000],
    y=[4.1, 4.1, 4.1],
    text=["0-7 %",
          "55-65%",
          "81-100%"],
    mode="text",
    name = "Color Areas:<br>% of WikiMatrix in training data"
))
#fig.add_trace(go.Scatter(x = [16500, 47300, 61100], y = [2.1,1.6,1.7]))
#fig.update_xaxes(type="log")

fig.update_layout(title_text="Test-Loss comparison over increasing training samples",
                 xaxis_title="Training samples",
                 yaxis_title="Test-Loss",
                 yaxis = dict(range=[0,5]),
                shapes=[
        # 1st highlight during Feb 4 - Feb 6
        dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=round_stats_model.total_samples[0],
            y0=0,
            x1=round_stats_model.total_samples[5],
            y1=1,
            fillcolor="darkgrey",
            opacity=0.5,
            layer="below",
            line_width=0,
        ),
        dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=45000,
            y0=0,
            x1=50000,
            y1=1,
            fillcolor="darkturquoise",
            opacity=0.5,
            layer="below",
            line_width=0,
    ),
        dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=58000,
            y0=0,
            x1=63000,
            y1=1,
            fillcolor="darkgrey",
            opacity=0.5,
            layer="below",
            line_width=0,
    )      
                ]
                 
                 
                 
                 )
fig.show()

fig.write_image("img/test_loss.png", width=800, height=350, scale=1.5)


In [26]:
loss_comparison = pd.DataFrame(
                                {"model":((round_stats_model.test_loss -  round_stats_baseline.test_loss)),
                                "baseline":((round_stats_baseline.test_loss - round_stats_baseline.test_loss)),
                                "samples": round_stats_model.total_samples})




In [34]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=loss_comparison.samples,y=loss_comparison.model, name= "Self-Learning Model"))
fig.add_trace(go.Scatter(x=loss_comparison.samples,y=loss_comparison.baseline, name = "Random-Pick Baseline Model"))
fig.add_trace(go.Scatter(
    x=[18200, 47500, 61000],
    y=[0.3, 0.3, 0.3],
    text=["0-7 %",
          "55-65%",
          "81-100%"],
    mode="text",
    name = "Color Areas:<br>% of WikiMatrix in training data"))
    
    
fig.update_layout(title_text="Test-Loss difference over increasing training samples",
                 xaxis_title="Training samples",
                 yaxis_title="Difference in loss",
                shapes=[
        # 1st highlight during Feb 4 - Feb 6
        dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=round_stats_model.total_samples[0],
            y0=0,
            x1=round_stats_model.total_samples[5],
            y1=1,
            fillcolor="darkgrey",
            opacity=0.5,
            layer="below",
            line_width=0,
        ),
        dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=45000,
            y0=0,
            x1=50000,
            y1=1,
            fillcolor="darkturquoise",
            opacity=0.5,
            layer="below",
            line_width=0,
    ),
        dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=58000,
            y0=0,
            x1=63000,
            y1=1,
            fillcolor="darkgrey",
            opacity=0.5,
            layer="below",
            line_width=0,
    )      
                ]
                 )
                 
fig.show()


fig.write_image("img/test_loss_difference.png", width=800, height=350, scale=1.5)


In [25]:
fig = go.Figure([go.Bar(x=["Tatoeba","WikiMatrix-Selection", "Tatoeba +<br>WikiMatrix-Selection", "Self-Learning"], 
                        y=[15.35,17.31,32, 34])])
fig.update_layout(title_text="BLEU-Score of models trained on different datasets",
                 xaxis_title="Dataset",
                 yaxis_title="BLEU-Score")
fig.show()
fig.write_image("img/bleu_score.png", width=800, height=550, scale=1)


In [9]:
bleu_comparison = pd.DataFrame(
                                {"model":(100*(round_stats_model.test_bleu -  round_stats_baseline.test_bleu)),
                                "baseline":(100*(round_stats_baseline.test_bleu - round_stats_baseline.test_bleu)),
                                "samples": round_stats_model.total_samples})
bleu_comparison

,model,baseline,samples
0,NaN,NaN,15808
1,NaN,NaN,15936
2,NaN,NaN,16192
3,NaN,NaN,16640
4,NaN,NaN,17664
5,NaN,NaN,19648
6,NaN,NaN,23680
7,NaN,NaN,31680
8,NaN,NaN,40640
9,NaN,NaN,47296


In [11]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=bleu_comparison.samples,y=bleu_comparison.model, name= "Learning Model"))
fig.add_trace(go.Scatter(x=bleu_comparison.samples,y=bleu_comparison.baseline, name = "Baseline Model"))
fig.update_xaxes(type="log")

fig.update_layout(title_text="BLEU-Score comparison of Learning Model and Baseline Model",
                 xaxis_title="Training samples",
                 yaxis_title="Difference [%-points]")
fig.show()

In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=round_stats_model.total_samples, y=round_stats_model.test_loss, name="Test loss"))
fig.add_trace(go.Scatter(x=round_stats_baseline.total_samples, y=round_stats_baseline.test_loss, name="Test loss baseline"))
fig.update_xaxes(type="log")
fig.update_layout(title_text="Test loss over increasing training size")

fig.show()